Please run the cell below for the runtime of each notebook. It will import necessary packages that you require to go through this tutorial.

In [ ]:
# --- Basic Setup (always run this first) ---

# Install dependencies 
%pip install -q gdown
%pip install -q git+https://github.com/Yuan-fang/fMRI-tutorial.git

# Import essential packages
import warnings
from pathlib import Path
import shutil
import module
from nilearn import image, plotting
from nilearn.image import index_img
from bids import BIDSLayout
import numpy as np
import os
import matplotlib.pyplot as plt

from tutorial.utils.paths import PathManager
from tutorial.utils.fetch import fetch_dataset

warnings.filterwarnings("ignore")


We also need to set up data directories. You can change to other directories and names according to your preference. But for this tutorial, let's stick with the same data directories.

In [ ]:
# --- Set up data directories ---

DATASET = "Haxby2001" # name of the dataset
BASE_DIR = Path.home() / "fmri_tutorial" # base directory for the tutorial
DATA_DIR = BASE_DIR / "data" / DATASET # data directory for the dataset
DERIV_DIR = BASE_DIR / "data" / "derivatives" # derivatives directory for processed data

for p in (DATA_DIR, DERIV_DIR):
    p.mkdir(parents=True, exist_ok=True)

In [ ]:
# --- Download dataset if not already present ---

# Google Drive link to the dataset
download_url = "https://drive.google.com/uc?id=1fPjbWhY6ZDOGSm59duKmOcCpgp5Zf5tX"
fetch_dataset(download_url, DATA_DIR)